# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [8]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Apps > Advanced app settings > App execution aliases.


ModuleNotFoundError: No module named 'langchain_elasticsearch'

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [2]:
!pip install python-dotenv
!pip install langchain-openai langchain-elasticsearch python-dotenv

import sys
print(sys.executable)  # sanity check: shows the Python your notebook is using
!{sys.executable} -m pip install -U langchain langchain-elasticsearch langchain-openai tiktoken python-dotenv jq lark



c:\Users\katyd\anaconda3\envs\mon_env\python.exe


In [4]:
import os, re, base64
from dotenv import load_dotenv

# If you've edited .env in the same session, allow overriding cached env
load_dotenv(override=True)

cid = os.getenv("ELASTIC_CLOUD_ID")
print("repr:", repr(cid))
print("length:", 0 if cid is None else len(cid))

# Basic shape check: "name:base64"
if not cid or ":" not in cid:
    raise ValueError("ELASTIC_CLOUD_ID missing colon or empty")

name, enc = cid.split(":", 1)
print("deployment name:", name)

# Quick sanity: the encoded part should contain at least one '$'
if "$" not in enc:
    print("WARNING: Encoded part has no '$' – likely truncated or not a real Cloud ID")

# Optional: try base64 decode to smoke-test (Cloud ID is URL-safe base64)
try:
    missing = (-len(enc)) % 4  # pad to multiple of 4
    decoded = base64.urlsafe_b64decode(enc + ("=" * missing))
    print("base64-decode OK, decoded len:", len(decoded))
except Exception as e:
    print("base64-decode FAILED:", e)


repr: 'Elasticsearch1:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ0NWVhZDBhYjFjYTg0MjUwOTllMWZjZmU2NWNjYjQwMCQ2YTg3NWEzNzRkZjM0OGY1ODNkZmE1OTgxOGQyOWYwOA=='
length: 147
deployment name: Elasticsearch1
base64-decode OK, decoded len: 97


In [5]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment
ELASTIC_CLOUD_ID = os.getenv("ELASTIC_CLOUD_ID")
ELASTIC_API_KEY = os.getenv("ELASTIC_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Optional check (just prints True/False, not the secrets themselves)
print("Cloud ID loaded:", ELASTIC_CLOUD_ID is not None)
print("API Key loaded:", ELASTIC_API_KEY is not None)
print("OpenAI Key loaded:", OPENAI_API_KEY is not None)

# Create embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Connect to Elasticsearch
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="search-ousy",  # use your existing index
    embedding=embeddings,
)


Cloud ID loaded: True
API Key loaded: True
OpenAI Key loaded: True


In [8]:
from getpass import getpass
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="Myindex1",  # give it a meaningful name
    embedding=embeddings,
)

## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [9]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [11]:
import sys
!{sys.executable} -m pip install -U langchain langchain-community tiktoken jq python-dotenv


In [2]:
# If you're on newer LangChain, prefer these imports:
# from langchain_community.document_loaders import JSONLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    # Safely populate metadata from the JSON object
    metadata["name"] = record.get("name")
    metadata["summary"] = record.get("summary")
    metadata["url"] = record.get("url")
    metadata["category"] = record.get("category")
    metadata["updated_at"] = record.get("updated_at")
    return metadata


loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",        # iterate each item if your file is a list
    content_key="content",  # make sure each record has a 'content' field
    metadata_func=metadata_func,
)

# EITHER token-aware splitter:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,         # any positive int
    chunk_overlap=200,      # must be < chunk_size
)

# OR simple character splitter:
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=800,
#     chunk_overlap=200,
# )

docs = loader.load_and_split(text_splitter=text_splitter)
print(f"Loaded {len(docs)} chunks")
print(docs[0].page_content[:200], "…")
print(docs[0].metadata)


Loaded 15 chunks
Effective: March 2020
Purpose

The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and produc …
{'source': 'C:\\Users\\katyd\\Downloads\\lab-chatbot-with-multi-query-retriever\\temp.json', 'seq_num': 1, 'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'url': './sharepoint/Work from home policy.txt', 'category': 'teams', 'updated_at': '2020-03-01'}


### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [5]:
# --- Build `docs` from a JSON file (temp.json) ---

# Compatible imports across LangChain versions
try:
    from langchain_community.document_loaders import JSONLoader
except ImportError:
    from langchain.document_loaders import JSONLoader

try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    from langchain.text_splitter import RecursiveCharacterTextSplitter

from pathlib import Path

# If you already defined metadata_func earlier, you can reuse it; otherwise use a safe default:
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["name"] = record.get("name")
    metadata["summary"] = record.get("summary")
    metadata["url"] = record.get("url")
    metadata["category"] = record.get("category")
    metadata["updated_at"] = record.get("updated_at")
    return metadata

json_path = Path("temp.json")
if not json_path.exists():
    raise FileNotFoundError("temp.json not found in the current working directory.")

loader = JSONLoader(
    file_path=str(json_path),
    jq_schema=".[]",          # iterate items if the file is a list
    content_key="content",    # each JSON object must have this field
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,           # positive integers required
    chunk_overlap=200,        # < chunk_size
)

docs = loader.load_and_split(text_splitter=text_splitter)
print(f"Loaded {len(docs)} chunks")



Loaded 15 chunks


# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [2]:
# assumes OPENAI_API_KEY / ELASTIC_* loaded and `docs` already exist
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")
INDEX_NAME = "my_docs_index"  # lowercase

vectorstore = ElasticsearchStore.from_documents(
    docs,
    embedding=embeddings,
    index_name=INDEX_NAME,
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)


NameError: name 'OPENAI_API_KEY' is not defined

In [10]:
# --- Ensure env + vectorstore exist ---
import os
from dotenv import load_dotenv, find_dotenv
env_path = find_dotenv()
if env_path:
    load_dotenv(env_path, override=False)

try:
    vectorstore
except NameError as e:
    raise RuntimeError(
        "`vectorstore` is not defined. Run the cell where you created "
        "ElasticsearchStore.from_documents(...) before this one."
    ) from e

# --- LLM + Retriever ---
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Missing OPENAI_API_KEY in your environment/.env")

llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)

# Use MultiQuery; switch to the simple retriever if you want fewer moving parts:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# --- Prompts & helpers (safe to re-run) ---
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

def _combine_documents(docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"):
    rendered = []
    for d in docs:
        # Fallback if metadata lacks "name"
        if "name" not in d.metadata:
            d.metadata.setdefault("name", d.metadata.get("source", "Unknown"))
        rendered.append(format_document(d, document_prompt))
    return document_separator.join(rendered)

# --- Runnable graph: {context, question} -> prompt -> llm -> str ---
_context = RunnableParallel(
    context=retriever | RunnableLambda(_combine_documents),
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm | StrOutputParser()

# --- Invoke ---
ans = chain.invoke("what is the nasa sales team?")
print("---- Answer ----")
print(ans)

# Optional quick checks:
# hits = vectorstore.similarity_search("nasa", k=3)
# print("hits:", len(hits), [h.metadata.get("name") or h.metadata.get("source") for h in hits])



RuntimeError: `vectorstore` is not defined. Run the cell where you created ElasticsearchStore.from_documents(...) before this one.

In [8]:
# --- make sure llm + retriever exist BEFORE building the chain ---
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)

# Simple retriever (works fine):
# retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# Or MultiQuery retriever (expands queries via the LLM):
retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)


NameError: name 'vectorstore' is not defined

In [1]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

NameError: name 'retriever' is not defined

**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?